<a href="https://colab.research.google.com/github/juanpasaflipz/build_agent_LangChain/blob/main/Agente_LangChan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente AI
Por sí mismos, los modelos de lenguaje no pueden realizar acciones; solo generan texto. Un caso de uso importante para LangChain es la creación de agentes.

Los agentes son sistemas que utilizan un LLM como motor de razonamiento para determinar qué acciones tomar y cuáles deben ser las entradas para esas acciones.

Los resultados de esas acciones pueden luego ser retroalimentados al agente, y este determinará si se necesitan más acciones o si está bien terminar.

En este tutorial, construiremos un agente que puede interactuar con múltiples herramientas diferentes: una siendo una base de datos local, y la otra un motor de búsqueda. Podrá hacer preguntas a este agente, verlo usar herramientas y tener conversaciones con él.

### Instalación
Esto instalará los requisitos mínimos de LangChain. Gran parte del valor de LangChain proviene de integrarlo con varios proveedores de modelos, almacenes de datos, etc.

Por defecto, las dependencias necesarias para hacer eso NO están instaladas. Tendrás que instalar las dependencias para integraciones específicas por separado.

In [ ]:
!pip install langchain

## LangSmith

Muchas de las aplicaciones que se construyen con LangChain contienen pasos con múltiples invocaciones de llamadas a LLM. A medida que estas aplicaciones se vuelvan cada vez más complejas, se vuelve crucial poder inspeccionar exactamente lo que está sucediendo dentro de su cadena o agente. **La mejor manera de hacer esto es con LangSmith**

In [ ]:
import getpass
import os

from google.colab import userdata;
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Definir la herramientas **(Tools)**

### Tavily

Una herramienta integrada en LangChain para usar fácilmente el motor de búsqueda Tavily como herramienta. *Ten en cuenta que esto requiere una clave de API*.

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Set the API key as an environment variable
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Create an instance of TavilySearchResults
search = TavilySearchResults()

api_key = os.environ['TAVILY_API_KEY']

# Use the TavilySearchResults with the API key
tavily_search = TavilySearchResults(tavily_api_key=api_key, max_results=2)

In [ ]:
# Perform a search query
search_results = tavily_search.invoke("quién se encuentra arriba en las encuestas para las elecciones presicenciales de México 2024?")

# Format and print the results in a human-readable way
def print_human_readable_results(results):
    for i, result in enumerate(results, 1):
        print(f"Result {i}: \n")
        for key, value in result.items():
            print(f"  {key}: {value}")
        print("\n" + "-"*40 + "\n")

print_human_readable_results(search_results)

In [ ]:
from IPython.display import display, Markdown

# Perform a search query
search_results = tavily_search.invoke("quién se encuantra arriba en las encuestas para las elecciones presicenciales de México 2024?")

"""
 def print_markdown_table(results):
    if not results:
        print("Sin resultados.")
        return

    # Collect all keys to ensure a consistent table structure
    all_keys = set()
    for result in results:
        all_keys.update(result.keys())

    all_keys = sorted(all_keys)  # Sort keys for consistent order

    # Print table header
    header = "| " + " | ".join(all_keys) + " |"
    separator = "| " + " | ".join(['---'] * len(all_keys)) + " |"
    print(header)
    print(separator)

    # Print table rows
    for result in results:
        row = "| " + " | ".join(str(result.get(key, "")) for key in all_keys) + " |"
        print(row)
    print("\n")
"""

#print_markdown_table(search_results) “””

# Format the search results as a Markdown table
def format_markdown_table(results):
    if not results:
        return "No results found."

    all_keys = set()
    for result in results:
        all_keys.update(result.keys())

    all_keys = sorted(all_keys)

    header = "| " + " | ".join(all_keys) + " |"
    separator = "| " + " | ".join(['---'] * len(all_keys)) + " |"
    rows = []

    for result in results:
        row = "| " + " | ".join(str(result.get(key, "")) for key in all_keys) + " |"
        rows.append(row)

    return "\n".join([header, separator] + rows)

# Format the search results as a Markdown table
search_result = format_markdown_table(search_results)

# Display the search results as Markdown
display(Markdown(search_result))

### Retriever
*Herramienta #2*

También crearemos un retriever (recuperador) sobre algunos datos propios.

In [ ]:
!pip install langchain_openai
!pip install openai
!pip install faiss-gpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("how to upload a dataset")[0]

Ahora que hemos poblado nuestro índice sobre el cual realizaremos la recuperación, podemos convertirlo fácilmente en una herramienta (el formato necesario para que un agente lo use correctamente).

https://api.python.langchain.com/en/latest/tools/langchain_core.tools.create_retriever_tool.html

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

## Tools (Herramientas)

In [ ]:
tools = [search, retriever_tool]

## Utilizando Modelos de Lenguaje


*   OpenAI    `pip install -qU langchain-openai `
*   Anthropic `pip install -qU langchain-anthropic `
*   Google `pip install -qU langchain-google-vertexai `
*   MistralAI `pip install -qU langchain-mistralai`
*   y otros....



In [ ]:
!pip install -qU langchain-openai

In [ ]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

Puedes llamar el modelo de lenguaje pasando una lista de mensajes. Por defecto, la respuesta es una cadena de contenido."

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

Ahora podemos ver cómo es habilitar que este modelo realice llamadas a herramientas. Para habilitar eso, usamos .bind_tools para darle al modelo de lenguaje conocimiento de estas herramientas.

In [ ]:
model_with_tools = model.bind_tools(tools)

Ahora podemos llamar el modelo. Primero, llamémoslo con un mensaje normal y veamos cómo responde. Podemos observar tanto el campo de contenido como el campo de llamadas a herramientas.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Ahora, intentemos llamarlo con una entrada que espere que se llame a una herramienta.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="quién se encuantra arriba en las encuestas para las elecciones presicenciales de México 2024?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

¡Podemos ver que ahora no hay contenido, pero sí hay una llamada a una herramienta! Quiere que llamemos a la herramienta Tavily Search.

*Esto aún no está llamando a esa herramienta; solo nos lo está indicando. Para llamarla, necesitamos crear nuestro agente.*

# Crear el agente
Ahora que hemos definido las herramientas y el LLM, podemos crear el agente. **Usaremos LangGraph para construir el agente**.

Actualmente estamos utilizando una interfaz de alto nivel para construir el agente, pero lo bueno de LangGraph es que esta interfaz de alto nivel está respaldada por una API de bajo nivel altamente controlable en caso de que desee modificar la lógica del agente.  --- *Esto quiere decir que en este momento se está utilizando una interfaz sencilla y fácil de usar para crear el agente. Sin embargo, LangGraph ofrece la ventaja de que esta interfaz sencilla está soportada por una API más avanzada y detallada, que permite un mayor control y personalización si se desea modificar el funcionamiento interno del agente.*

Ahora, podemos inicializar el agente con el LLM y las herramientas.

Ten en cuenta que solamente estamos pasando el modelo, no el modelo con herramientas. **Eso es porque create_tool_calling_executor llamará a .bind_tools por nosotros automáticamente**.

In [ ]:
!pip install langgraph

In [ ]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)

# Ejecutar el agente
¡Ahora podemos ejecutar el agente en algunas consultas! Ten en cuenta que, por ahora, todas son consultas sin estado (no recordará interacciones previas).

Nota que el agente devolverá el estado final al final de la interacción (lo que incluye cualquier entrada; más adelante veremos cómo obtener solo las salidas).

Primero, veamos cómo responde cuando no hay necesidad de llamar a una herramienta:

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

*Para ver exactamente lo que está sucediendo internamente (y para asegurarnos de que no está llamando a una herramienta), podemos echar un vistazo a [LangSmith Trace](https://smith.langchain.com/public/r)*

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="quién se encuantra arriba en las encuestas para las elecciones presicenciales de México 2024?")]}
)
response["messages"]

## Transmisión de mensajes (Streaming)
Hemos visto cómo se puede llamar al agente con .invoke para obtener una respuesta final. Si el agente está ejecutando múltiples pasos, eso puede tardar un tiempo. Para mostrar el progreso intermedio, podemos transmitir los mensajes a medida que ocurren.

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="quién se encuentra arriba en las encuestas para las elecciones presicenciales de México 2024?")]}
):
    print(chunk)
    print("----")

## Transmisión de tokens
Además de transmitir mensajes, también es útil transmitir tokens. Podemos hacer esto con el método .astream_events.

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="quién se encuentra arriba en las encuestas para las elecciones presicenciales de México 2024?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

## Agregar memoria
Como mencioné anteriormente, este agente no tiene estado, *es stateless*. Esto significa que no recuerda interacciones previas. Para darle memoria, necesitamos pasar un checkpointer. Al pasar un checkpointer, también tenemos que pasar un `thread_id` al invocar al agente (para que sepa de qué hilo/conversación debe reanudar).

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hola soy juan!")]}, config
):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")